In [1]:
%load_ext autoreload

In [2]:
import os

import hydra
import lightning
import wandb
from openretina.data_io.cyclers import LongCycler, ShortCycler

/user/vystrcilova/u14647/.conda/envs/venv_or/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
with hydra.initialize(config_path="../configs", version_base="1.3"):
    cfg = hydra.compose(config_name="vystrcilova_2024_nm_cnn.yaml")
print(cfg)
cfg.dataloader.num_of_frames = cfg.model.temporal_kernel_sizes[0]
cfg.dataloader.num_of_layers = len(cfg.model.temporal_kernel_sizes)
dataloaders = hydra.utils.instantiate(cfg.dataloader)
n_neurons_dict = {}
for index in cfg.dataloader.files.keys():
    n_neurons_dict[index] = dataloaders['train'][index].dataset.n_neurons


{'data_io': {'basepath_nm': '/projects/extern/nhr/nhr_ni/nim00010/dir.project/marmoset_data/', 'cache_dir': '/projects/extern/nhr/nhr_ni/nim00010/dir.project/', 'response_path': 'responses', 'image_path': 'stimuli_padded_4/', 'stas_path': 'stas', 'sta_file': 'cell_data_01_WN_stas_cell'}, 'dataloader': {'_target_': 'openretina.data_io.sridhar_2025.dataloaders.frame_movie_loader', '_convert_': 'object', 'basepath': '/projects/extern/nhr/nhr_ni/nim00010/dir.project/marmoset_data/', 'files': {'01': 'responses/cell_responses_01_fixation_movie.pkl', '02': 'responses/cell_responses_02_fixation_movie.pkl', '03': 'responses/cell_responses_03_fixation_movie.pkl', '04': 'responses/cell_responses_04_fixation_movie.pkl'}, 'fixation_files': {'01': 'fixations/complete_fixations_seed_2022and2023_s4_fixed.txt', '02': 'fixations/complete_fixations_seed2022_s4.txt', '03': 'fixations/complete_fixations_seed_2022and2023_s4_fixed.txt', '04': 'fixations/complete_fixations_seed_2022and2023_s4_fixed.txt'}, 'bi

100%|██████████| 14544/14544 [01:33<00:00, 155.17it/s]


Random seed 1000 has been set.
train idx: [10 13 17 16  8  7  6 11  1 14  5  0  9  3 19 15]
val idx: [12 18 20  2  4]
Trial separation for 01
training trials:  16 [10 13 17 16  8  7  6 11  1 14  5  0  9  3 19 15]
validation trials:  5 [12 18 20  2  4] 

Random seed 1000 has been set.
train idx: [7 6 9 1 2 8 0 3]
val idx: [ 4 10  5]
Trial separation for 02
training trials:  8 [7 6 9 1 2 8 0 3]
validation trials:  3 [ 4 10  5] 

Random seed 1000 has been set.
train idx: [16 11 18 10 12  5  6 17  9 19  0  2  7 14 15 13]
val idx: [3 1 8 4]
Trial separation for 03
training trials:  16 [16 11 18 10 12  5  6 17  9 19  0  2  7 14 15 13]
validation trials:  4 [3 1 8 4] 

Random seed 1000 has been set.
train idx: [16 11 18 10 12  5  6 17  9 19  0  2  7 14 15 13]
val idx: [3 1 8 4]
Trial separation for 04
training trials:  16 [16 11 18 10 12  5  6 17  9 19  0  2  7 14 15 13]
validation trials:  4 [3 1 8 4] 



In [4]:
input_shape = next(iter(dataloaders['train']['01']))
cfg.model['in_shape'] = list(input_shape[0].shape[1:])
cfg.model['n_neurons_dict'] = n_neurons_dict
print(cfg.model.in_shape)

[1, 134, 80, 90]


In [5]:
model = hydra.utils.instantiate(cfg.model)
batch = next(iter(dataloaders['train']['01']))
print('input img shape', batch[0].shape)
print('response shape', batch[1].shape)

/scratch-grete/projects/nim00010/open-retina/openretina/modules/readout/base.py:62: UserWarning: Readout is NOT initialized with mean activity but with 0!
  warnings.warn("Readout is NOT initialized with mean activity but with 0!")


input img shape torch.Size([16, 1, 134, 80, 90])
response shape torch.Size([16, 100, 370])


In [ ]:
%autoreload

In [6]:
model(batch[0])

tensor([[[0.7525, 0.6297, 0.7903,  ..., 0.6786, 0.8411, 0.7162],
         [0.9230, 0.6043, 0.8601,  ..., 0.7616, 0.7039, 0.6800],
         [0.7813, 0.6729, 0.7828,  ..., 0.8154, 0.7412, 0.7660],
         ...,
         [0.8298, 0.5970, 0.8077,  ..., 0.9255, 0.8073, 0.7437],
         [0.8285, 0.7388, 0.7605,  ..., 0.8136, 0.8166, 0.9623],
         [0.6986, 0.7408, 0.8166,  ..., 0.8063, 0.9615, 0.7313]],

        [[0.8942, 0.8192, 0.9829,  ..., 0.7128, 0.6606, 0.8789],
         [0.7369, 0.8318, 0.7846,  ..., 0.9193, 0.9106, 0.7390],
         [0.7191, 0.6708, 0.8480,  ..., 0.9810, 0.6764, 0.8480],
         ...,
         [0.9009, 0.7809, 0.8225,  ..., 0.9069, 0.7218, 0.7831],
         [0.7757, 0.7231, 0.7791,  ..., 0.6975, 0.8262, 0.7921],
         [0.7229, 0.7438, 0.7891,  ..., 0.7783, 0.8071, 0.7521]],

        [[0.9103, 0.8410, 0.8151,  ..., 0.7417, 0.8177, 0.8304],
         [0.8411, 0.8550, 0.8510,  ..., 0.8496, 0.7439, 0.9647],
         [0.8265, 0.9203, 0.8357,  ..., 0.8402, 0.6825, 1.

In [7]:
log_save_path = os.path.join('/projects/extern/nhr/nhr_ni/nim00010/dir.project/logs', "cnns_nm/")
os.makedirs(log_save_path, exist_ok=True, )

logger = lightning.pytorch.loggers.WandbLogger(
    name="",
    save_dir=log_save_path,
)
early_stopping = lightning.pytorch.callbacks.EarlyStopping(
    monitor="val_correlation",
    patience=10,
    mode="max",
    verbose=False,
    min_delta=0.001,
)

lr_monitor = lightning.pytorch.callbacks.LearningRateMonitor(logging_interval="epoch")

model_checkpoint = lightning.pytorch.callbacks.ModelCheckpoint(
    monitor="val_correlation", mode="max", save_weights_only=False
)

trainer = lightning.Trainer(max_epochs=100, logger=logger, callbacks=[early_stopping, lr_monitor, model_checkpoint],
                            accelerator="gpu", log_every_n_steps=10)

/user/vystrcilova/u14647/.conda/envs/venv_or/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /user/vystrcilova/u14647/.conda/envs/venv_or/lib/pyt ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [8]:
train_loader = LongCycler(dataloaders["train"])
val_loader = LongCycler(dataloaders["validation"])

In [9]:
trainer.fit(model, train_loader, val_loader)


You are using a CUDA device ('NVIDIA A100-SXM4-80GB MIG 1g.10gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: m_vys (retinal-circuit-modeling) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                               | Params | Mode 
--------------------------------------------------------------------------------
0 | core             | SimpleCoreWrapper                  | 16.1 K | train
1 | readout          | MultiSampledGaussianReadoutWrapper | 26.6 K | train
2 | loss             | PoissonLoss3d                      | 0      | train
3 | correlation_loss | CorrelationLoss3d                  | 0      | train
--------------------------------------------------------------------------------
42.7 K    Trainable params
0         Non-trainable params
42.7 K    Total params
0.171     Total estimated model params size (MB)
19        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
wandb.finish()

In [ ]:
import os

import hydra
import lightning
import wandb
from openretina.data_io.cyclers import LongCycler, ShortCycler

In [ ]:
    with hydra.initialize(config_path="configs", version_base="1.3"):
        cfg = hydra.compose(config_name="vystrcilova_2024_nm_cnn.yaml")
    print(cfg)
    cfg.dataloader.num_of_frames = cfg.model.temporal_kernel_sizes[0]
    cfg.dataloader.num_of_layers = len(cfg.model.temporal_kernel_sizes)
    dataloaders = hydra.utils.instantiate(cfg.dataloader)
    n_neurons_dict = {}
    for index in cfg.dataloader.files.keys():
        n_neurons_dict[index] = dataloaders['train'][index].dataset.n_neurons

    input_shape = next(iter(dataloaders['train']['01']))
    cfg.model['in_shape'] = list(input_shape[0].shape[1:])
    cfg.model['n_neurons_dict'] = n_neurons_dict
    print(cfg.model.in_shape)

    model = hydra.utils.instantiate(cfg.model)
    batch = next(iter(dataloaders['train']['01']))
    print('input img shape', batch[0].shape)
    print('response shape', batch[1].shape)

    log_save_path = os.path.join('/projects/extern/nhr/nhr_ni/nim00010/dir.project/logs', "cnns_nm/")
    os.makedirs(log_save_path, exist_ok=True, )

    logger = lightning.pytorch.loggers.WandbLogger(
        name="",
        save_dir=log_save_path,
    )
    early_stopping = lightning.pytorch.callbacks.EarlyStopping(
        monitor="val_correlation",
        patience=10,
        mode="max",
        verbose=False,
        min_delta=0.001,
    )

    lr_monitor = lightning.pytorch.callbacks.LearningRateMonitor(logging_interval="epoch")

    model_checkpoint = lightning.pytorch.callbacks.ModelCheckpoint(
        monitor="val_correlation", mode="max", save_weights_only=False
    )

    trainer = lightning.Trainer(max_epochs=100, logger=logger, callbacks=[early_stopping, lr_monitor, model_checkpoint],
                                accelerator="gpu", log_every_n_steps=10)

    train_loader = LongCycler(dataloaders["train"])
    val_loader = LongCycler(dataloaders["validation"])

    trainer.fit(model, train_loader, val_loader)
    wandb.finish()

In [ ]:
    with hydra.initialize(config_path="configs", version_base="1.3"):
        cfg = hydra.compose(config_name="vystrcilova_2024_nm_cnn.yaml")
    print(cfg)
    cfg.dataloader.num_of_frames = cfg.model.temporal_kernel_sizes[0]
    cfg.dataloader.num_of_layers = len(cfg.model.temporal_kernel_sizes)
    dataloaders = hydra.utils.instantiate(cfg.dataloader)
    n_neurons_dict = {}
    for index in cfg.dataloader.files.keys():
        n_neurons_dict[index] = dataloaders['train'][index].dataset.n_neurons

    input_shape = next(iter(dataloaders['train']['01']))
    cfg.model['in_shape'] = list(input_shape[0].shape[1:])
    cfg.model['n_neurons_dict'] = n_neurons_dict
    print(cfg.model.in_shape)

    model = hydra.utils.instantiate(cfg.model)
    batch = next(iter(dataloaders['train']['01']))
    print('input img shape', batch[0].shape)
    print('response shape', batch[1].shape)

    log_save_path = os.path.join('/projects/extern/nhr/nhr_ni/nim00010/dir.project/logs', "cnns_nm/")
    os.makedirs(log_save_path, exist_ok=True, )

    logger = lightning.pytorch.loggers.WandbLogger(
        name="",
        save_dir=log_save_path,
    )
    early_stopping = lightning.pytorch.callbacks.EarlyStopping(
        monitor="val_correlation",
        patience=10,
        mode="max",
        verbose=False,
        min_delta=0.001,
    )

    lr_monitor = lightning.pytorch.callbacks.LearningRateMonitor(logging_interval="epoch")

    model_checkpoint = lightning.pytorch.callbacks.ModelCheckpoint(
        monitor="val_correlation", mode="max", save_weights_only=False
    )

    trainer = lightning.Trainer(max_epochs=100, logger=logger, callbacks=[early_stopping, lr_monitor, model_checkpoint],
                                accelerator="gpu", log_every_n_steps=10)

    train_loader = LongCycler(dataloaders["train"])
    val_loader = LongCycler(dataloaders["validation"])

    trainer.fit(model, train_loader, val_loader)
    wandb.finish()

In [ ]:
%load_ext autoreload


In [ ]:
with hydra.initialize(config_path=os.path.join("..", "configs"), version_base="1.3"):
    cfg = hydra.compose(config_name="vystrcilova_2024_nm_cnn.yaml")

In [ ]:
print(cfg)

In [ ]:
cfg.dataloader.num_of_frames = cfg.model.temporal_kernel_sizes[0]
cfg.dataloader.num_of_layers = len(cfg.model.temporal_kernel_sizes)

In [ ]:
cfg.dataloader.num_of_hidden_frames

In [ ]:
dataloaders = hydra.utils.instantiate(cfg.dataloader)

In [ ]:
cfg.dataloader.files

In [ ]:
n_neurons_dict = {}
for index in cfg.dataloader.files.keys():
    n_neurons_dict[index] = dataloaders['train'][index].dataset.n_neurons


In [ ]:
input_shape = next(iter(dataloaders['train']['01']))

In [ ]:
cfg.model['in_shape'] = list(input_shape[0].shape[1:])
cfg.model['n_neurons_dict'] = n_neurons_dict
print(cfg.model.in_shape)

In [ ]:
%autoreload

In [ ]:
model = hydra.utils.instantiate(cfg.model)

In [ ]:
model

In [ ]:
batch = next(iter(dataloaders['train']['01']))

In [ ]:
print('input img shape', batch[0].shape)
print('response shape', batch[1].shape)

In [ ]:
y = model(batch[0], '01')
print(y)

In [ ]:
log_save_path = os.path.join('/projects/extern/nhr/nhr_ni/nim00010/dir.project/logs', "cnns_nm/")
os.makedirs(log_save_path, exist_ok=True, )


logger = lightning.pytorch.loggers.WandbLogger(
    name="",
    save_dir=log_save_path,
)
early_stopping = lightning.pytorch.callbacks.EarlyStopping(
    monitor="val_correlation",
    patience=10,
    mode="max",
    verbose=False,
    min_delta=0.001,
)

lr_monitor = lightning.pytorch.callbacks.LearningRateMonitor(logging_interval="epoch")

model_checkpoint = lightning.pytorch.callbacks.ModelCheckpoint(
    monitor="val_correlation", mode="max", save_weights_only=False
)

trainer = lightning.Trainer(max_epochs=100, logger=logger, callbacks=[early_stopping, lr_monitor, model_checkpoint],
                            accelerator="gpu", log_every_n_steps=10)

In [ ]:
train_loader = LongCycler(dataloaders["train"])
val_loader = LongCycler(dataloaders["validation"])

In [ ]:
trainer.fit(model, train_loader, val_loader)


In [ ]:
wandb.finish()

In [ ]:
import wandb

In [35]:
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
with hydra.initialize(config_path=os.path.join("..", "configs"), version_base="1.3"):
    cfg = hydra.compose(config_name="vystrcilova_2024_nm_cnn.yaml")

In [4]:
print(cfg)

{'data_io': {'basepath_nm': '/projects/extern/nhr/nhr_ni/nim00010/dir.project/marmoset_data/', 'cache_dir': '/projects/extern/nhr/nhr_ni/nim00010/dir.project/', 'response_path': 'responses', 'image_path': 'stimuli_padded_4/', 'stas_path': 'stas', 'sta_file': 'cell_data_01_WN_stas_cell'}, 'dataloader': {'_target_': 'openretina.data_io.sridhar_2025.dataloaders.frame_movie_loader', '_convert_': 'object', 'basepath': '/projects/extern/nhr/nhr_ni/nim00010/dir.project/marmoset_data/', 'files': {'01': 'responses/cell_responses_01_fixation_movie.pkl', '02': 'responses/cell_responses_02_fixation_movie.pkl', '03': 'responses/cell_responses_03_fixation_movie.pkl', '04': 'responses/cell_responses_04_fixation_movie.pkl'}, 'fixation_files': {'01': 'fixations/complete_fixations_seed_2022and2023_s4_fixed.txt', '02': 'fixations/complete_fixations_seed2022_s4.txt', '03': 'fixations/complete_fixations_seed_2022and2023_s4_fixed.txt', '04': 'fixations/complete_fixations_seed_2022and2023_s4_fixed.txt'}, 'bi

In [16]:
cfg.dataloader.num_of_frames = cfg.model.temporal_kernel_sizes[0]
cfg.dataloader.num_of_layers = len(cfg.model.temporal_kernel_sizes)

In [17]:
cfg.dataloader.num_of_hidden_frames

5

In [18]:
dataloaders = hydra.utils.instantiate(cfg.dataloader)

Loading all frames from: /projects/extern/nhr/nhr_ni/nim00010/dir.project/marmoset_data/stimuli_padded_4 into memory


100%|██████████| 14544/14544 [01:02<00:00, 233.30it/s]


Random seed 1000 has been set.
train idx: [10 13 17 16  8  7  6 11  1 14  5  0  9  3 19 15]
val idx: [12 18 20  2  4]
Trial separation for 01
training trials:  16 [10 13 17 16  8  7  6 11  1 14  5  0  9  3 19 15]
validation trials:  5 [12 18 20  2  4] 

Random seed 1000 has been set.
train idx: [7 6 9 1 2 8 0 3]
val idx: [ 4 10  5]
Trial separation for 02
training trials:  8 [7 6 9 1 2 8 0 3]
validation trials:  3 [ 4 10  5] 

Random seed 1000 has been set.
train idx: [16 11 18 10 12  5  6 17  9 19  0  2  7 14 15 13]
val idx: [3 1 8 4]
Trial separation for 03
training trials:  16 [16 11 18 10 12  5  6 17  9 19  0  2  7 14 15 13]
validation trials:  4 [3 1 8 4] 

Random seed 1000 has been set.
train idx: [16 11 18 10 12  5  6 17  9 19  0  2  7 14 15 13]
val idx: [3 1 8 4]
Trial separation for 04
training trials:  16 [16 11 18 10 12  5  6 17  9 19  0  2  7 14 15 13]
validation trials:  4 [3 1 8 4] 



In [19]:
cfg.dataloader.files

{'01': 'responses/cell_responses_01_fixation_movie.pkl', '02': 'responses/cell_responses_02_fixation_movie.pkl', '03': 'responses/cell_responses_03_fixation_movie.pkl', '04': 'responses/cell_responses_04_fixation_movie.pkl'}

In [20]:
n_neurons_dict = {}
for index in cfg.dataloader.files.keys():
    n_neurons_dict[index] = dataloaders['train'][index].dataset.n_neurons


In [21]:
input_shape = next(iter(dataloaders['train']['01']))

In [22]:
cfg.model['in_shape'] = list(input_shape[0].shape[1:])
cfg.model['n_neurons_dict'] = n_neurons_dict
print(cfg.model.in_shape)

[1, 24, 80, 90]


In [33]:
%autoreload

In [24]:
model = hydra.utils.instantiate(cfg.model)

/scratch-grete/projects/nim00010/open-retina/openretina/modules/readout/base.py:62: UserWarning: Readout is NOT initialized with mean activity but with 0!
  warnings.warn("Readout is NOT initialized with mean activity but with 0!")


In [25]:
model

CoreGaussianReadout(
  (core): <super: <class 'Core'>, <SimpleCoreWrapper object>> [SimpleCoreWrapper regularizers: gamma_hidden = 0.0|gamma_in_sparse = 0.0|gamma_input = 0.0|gamma_temporal = 40.0]
  
  (readout): MultiSampledGaussianReadoutWrapper(
    (01): full FullGaussian2d (8 x 70 x 80 -> 370) with bias
    (02): full FullGaussian2d (8 x 70 x 80 -> 182) with bias
    (03): full FullGaussian2d (8 x 70 x 80 -> 495) with bias
    (04): full FullGaussian2d (8 x 70 x 80 -> 110) with bias
  )
  (loss): PoissonLoss3d()
  (correlation_loss): CorrelationLoss3d()
)

In [26]:
batch = next(iter(dataloaders['train']['01']))

In [27]:
print('input img shape', batch[0].shape)
print('response shape', batch[1].shape)

input img shape torch.Size([16, 1, 24, 80, 90])
response shape torch.Size([16, 10, 370])


In [32]:
y = model(batch[0], '01')
print(y)

tensor([[[-338.1945, -351.1412, -343.5503,  ..., -351.8071, -351.3875,
          -351.5482],
         [-338.1945, -351.1412, -343.5502,  ..., -351.8071, -351.3875,
          -351.5482],
         [-338.1945, -351.1412, -343.5503,  ..., -351.8071, -351.3875,
          -351.5482],
         ...,
         [-338.1945, -351.1412, -343.5503,  ..., -351.8071, -351.3875,
          -351.5482],
         [-338.1945, -351.1412, -343.5503,  ..., -351.8071, -351.3875,
          -351.5482],
         [-338.1945, -351.1412, -343.5503,  ..., -351.8071, -351.3875,
          -351.5482]],

        [[-338.1945, -351.1412, -343.5503,  ..., -351.8071, -351.3875,
          -351.5482],
         [-338.1945, -351.1411, -343.5502,  ..., -351.8071, -351.3875,
          -351.5482],
         [-338.1945, -351.1412, -343.5503,  ..., -351.8071, -351.3875,
          -351.5482],
         ...,
         [-338.1945, -351.1412, -343.5503,  ..., -351.8071, -351.3875,
          -351.5482],
         [-338.1945, -351.1412, -343.550

In [29]:
log_save_path = os.path.join('/projects/extern/nhr/nhr_ni/nim00010/dir.project/logs', "cnns_nm/")
os.makedirs(log_save_path, exist_ok=True, )


logger = lightning.pytorch.loggers.WandbLogger(
    name="",
    save_dir=log_save_path,
)
early_stopping = lightning.pytorch.callbacks.EarlyStopping(
    monitor="val_correlation",
    patience=10,
    mode="max",
    verbose=False,
    min_delta=0.001,
)

lr_monitor = lightning.pytorch.callbacks.LearningRateMonitor(logging_interval="epoch")

model_checkpoint = lightning.pytorch.callbacks.ModelCheckpoint(
    monitor="val_correlation", mode="max", save_weights_only=False
)

trainer = lightning.Trainer(max_epochs=100, logger=logger, callbacks=[early_stopping, lr_monitor, model_checkpoint],
                            accelerator="gpu", log_every_n_steps=10)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [30]:
train_loader = LongCycler(dataloaders["train"])
val_loader = LongCycler(dataloaders["validation"])

In [30]:
trainer.fit(model, train_loader, val_loader)


You are using a CUDA device ('NVIDIA A100-SXM4-40GB MIG 1g.10gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: m_vys (retinal-circuit-modeling) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                               | Params | Mode 
--------------------------------------------------------------------------------
0 | core             | SimpleCoreWrapper                  | 2.0 K  | train
1 | readout          | MultiSampledGaussianReadoutWrapper | 17.4 K | train
2 | loss             | PoissonLoss3d                      | 0      | train
3 | correlation_loss | CorrelationLoss3d                  | 0      | train
--------------------------------------------------------------------------------
19.3 K    Trainable params
0         Non-trainable params
19.3 K    Total params
0.077     Total estimated model params size (MB)
19        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...

KeyboardInterrupt


KeyboardInterrupt



In [ ]:
wandb.finish()

In [ ]:
import wandb